In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import mne
import os
import re
import torch
import pickle

from utils import *

In [2]:
task_dict = {
    'Resting state': {'annotation': 'T0', 'runs': ['03', '04', '05', '06','07', '08', '09', '10', '11', '12', '13', '14']},
    'Left fist, performed': {'annotation': 'T1', 'runs': ['03', '07' '11']},
    'Left fist, imagined': {'annotation': 'T1', 'runs': ['04', '08', '12']},
    'Both fists, performed': {'annotation': 'T1', 'runs': ['05', '09', '13']},
    'Both fists, imagined': {'annotation': 'T1', 'runs': ['06', '10', '14']},
    'Right fist, performed': {'annotation': 'T2', 'runs': ['03', '07', '11']},
    'Right fist, imagined': {'annotation': 'T2', 'runs': ['04', '08', '12']},
    'Both feet, performed': {'annotation': 'T2', 'runs': ['05', '09', '13']},
    'Both feet, imagined': {'annotation': 'T2', 'runs': ['06', '10', '14']}
}


label_dict = {
    'T0': {'03': 'Resting state', '04': 'Resting state', '05': 'Resting state', '06': 'Resting state', '07': 'Resting state', '08': 'Resting state', '09': 'Resting state', '10': 'Resting state', '11': 'Resting state', '12': 'Resting state', '13': 'Resting state', '14': 'Resting state'},
    'T1': {'03': 'Left fist, performed', '04': 'Left fist, imagined', '05': 'Both fists, performed', '06': 'Both fists, imagined', '07': 'Left fist, performed', '08': 'Left fist, imagined', '09': 'Both fists, performed', '10': 'Both fists, imagined', '11': 'Left fist, performed', '12': 'Left fist, imagined', '13': 'Both fists, performed', '14': 'Both fists, imagined'},
    'T2': {'03': 'Right fist, performed', '04': 'Right fist, imagined', '05': 'Both feet, performed', '06': 'Both feet, imagined', '07': 'Right fist, performed', '08': 'Right fist, imagined', '09': 'Both feet, performed', '10': 'Both feet, imagined', '11': 'Right fist, performed', '12': 'Right fist, imagined', '13': 'Both feet, performed', '14': 'Both feet, imagined'}
}

In [3]:
DATA_PATH_RAW = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (raw)/files/'
DATA_PATH_LABELLED = '/home/williamtheodor/Documents/DL for EEG Classification/data/eegmmidb (labelled)/'


# create folder for each label
for label in task_dict.keys():
    if not os.path.exists(DATA_PATH_LABELLED + label):
        os.makedirs(DATA_PATH_LABELLED + label)

In [5]:
NUMBER_SAMPLES = 1024
NUMBER_CHANNELS = 20

for patient_folder in tqdm(os.listdir(DATA_PATH_RAW)):
    for file in os.listdir(DATA_PATH_RAW + patient_folder + '/'):
        if file.endswith('.edf') and file[5:7] not in ['01', '02']:
            file_path = DATA_PATH_RAW + patient_folder + '/' + file

            raw = get_raw(file_path)
            annotations = get_annotations(file_path)

            annotation_dict = get_window_dict(raw, annotations)

            for annotation in annotation_dict.keys():
                for raw in annotation_dict[annotation]:

                    raw = pick_and_rename_channels(raw.copy())                    

                    try: 
                        x = np.zeros((1, NUMBER_CHANNELS, NUMBER_SAMPLES))
                        x[:,:19,:] = raw.copy().get_data()[:,:NUMBER_SAMPLES].reshape(1,NUMBER_CHANNELS-1,NUMBER_SAMPLES)
                        x[:,19,:] = np.ones((1, NUMBER_SAMPLES)) * -1  
                        x = torch.from_numpy(x).float()

                        label = label_dict[annotation][file[5:7]]
                        
                        picklePath = DATA_PATH_LABELLED + label + '/' + file[0:7] + '_' + annotation + '_' + label + '.pkl'
                        with open(picklePath, 'wb') as handle:
                            pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)
                    except:
                        print('Error with file: ' + file, raw.copy().get_data().shape)
                    

  4%|▎         | 4/109 [00:23<10:26,  5.97s/it]

Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R10.edf (1, 20, 1024)
Error with file: S088R14.edf (1, 20, 1024)
Error with file: S088R14.edf (1, 20, 1024)
Error with file: S088R14.edf (1, 20, 1024)
Error with file: S088R14.edf (1, 20, 1024)
Error with 

  5%|▍         | 5/109 [00:29<10:08,  5.85s/it]

Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)
Error with file: S088R11.edf (1, 20, 1024)


 46%|████▌     | 50/109 [05:06<06:11,  6.30s/it]/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_path, verbose=False, preload=True)
/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_path, verbose=False, preload=True)
/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_path, verbose=False, preload=True)
/home/williamtheodor/Documents/DL for EEG Classification/BENDR-XAI/notebooks/utils.py:27: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_file_pat

...

 71%|███████   | 77/109 [07:49<03:21,  6.30s/it]

Error with file: S106R05.edf (1, 20, 1024)


 82%|████████▏ | 89/109 [09:00<01:58,  5.94s/it]

Error with file: S104R08.edf (1, 20, 1024)


100%|██████████| 109/109 [11:02<00:00,  6.07s/it]
